Transform the C-GLOPS data to our reference raster CLC. 
Use gdal_translate -r average to calculate the average of the cells as they are % forrestcover. 
We also need to devide by 100

from man gdal_translate
average computes the average of all non-NODATA contributing pixels. Starting with GDAL 3.1, this is a weighted average  taking  into  ac‐
              count properly the weight of source pixels not contributing fully to the target pixel.

We also need to recode the NA values to 0, such that the average gets calculated correctly. 



In [9]:
!gdalinfo in_data/C-GLOPS/2019/Tree_CoverFraction_2019_clipped.tif | grep NoD


  NoData Value=255


In [13]:
%%bash

for i in {1..100}; do
    # print the current number and a decimal value from 0.01 to 1 in the output file
    printf "%d %.2f\n" "$i" "$(echo "scale=2;$i/100"|bc)" 
done > /tmp/recode4bash.txt 

# add the last line to the output file
echo "255 0" >> /tmp/recode4bash.txt 


In [15]:
!tail /tmp/recode4bash.txt 

92 0.92
93 0.93
94 0.94
95 0.95
96 0.96
97 0.97
98 0.98
99 0.99
100 1.00
255 0


In [18]:
%%bash
start=`date +%s`

pkreclass -co COMPRESS=DEFLATE -co ZLEVEL=9  \
-code /tmp/recode4bash.txt                   \
-nodata 0                                    \
-ot Float64                                  \
-co BIGTIFF=YES                              \
-i in_data/C-GLOPS/2019/Tree_CoverFraction_2019_clipped.tif                                \
-o in_data/C-GLOPS/2019/Tree_CoverFraction_2019_clipped_0nodata.tif                        \

end=`date +%s`

expr $end - $start

0...10...20...30...40...50...60...70...80...90...100 - done.
2118


Next step is to transform it to the reference raster

In [17]:
!gdalinfo in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif

Driver: GTiff/GeoTIFF
Files: in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif
       in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif.aux.xml
Size is 65000, 46000
Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
Coordinate System is:
PROJCRS["ETRS_1989_LAEA",
    BASEGEOGCRS["ETRS89",
        DATUM["European Terrestrial Reference System 1989",
            ELLIPSOID["GRS 1980",6378137,298.257222101004,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4258]],
    CONVERSION["Lambert Azimuthal Equal Area",
     

In [21]:
%%bash 
gdalwarp -co COMPRESS=DEFLATE -co ZLEVEL=9 \
-overwrite                                 \
-t_srs EPSG:3035                           \
-tr 100 100                                \
-r average                                 \
-te 900000 900000 7400000 5500000          \
-co BIGTIFF=YES                              \
-tap                                       \
in_data/C-GLOPS/2019/Tree_CoverFraction_2019_clipped_0nodata.tif   \
out_data/C-GLOPS/2019/Tree_CoverFraction_2019_mean100m_0nodata.tif

Creating output file that is 65000P x 46000L.
Processing in_data/C-GLOPS/2019/Tree_CoverFraction_2019_clipped_0nodata.tif [1/1] : 0Using internal nodata values (e.g. 0) for image in_data/C-GLOPS/2019/Tree_CoverFraction_2019_clipped_0nodata.tif.
Copying nodata values from source in_data/C-GLOPS/2019/Tree_CoverFraction_2019_clipped_0nodata.tif to destination out_data/C-GLOPS/2019/Tree_CoverFraction_2019_mean100m_0nodata.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [2]:
!gdalinfo out_data/C-GLOPS/2019/Tree_CoverFraction_2019_mean100m_0nodata.tif

Driver: GTiff/GeoTIFF
Files: out_data/C-GLOPS/2019/Tree_CoverFraction_2019_mean100m_0nodata.tif
Size is 65000, 46000
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference Frame 2014"],
       